In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda


In [6]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [8]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 100)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

model_path = 'best_teacher_model.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [10]:
teacher_accuracy = evaluate(teacher, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")

Teacher Model Test Accuracy: 80.89%


In [11]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 100)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 81.12%


In [14]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [15]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [16]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [17]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [18]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [19]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [20]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [21]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [23]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [40]:

student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
)


Epoch 1/50 | Train Loss: 9.5556 | Train Acc: 54.33% | Val Acc: 61.88%
Epoch 2/50 | Train Loss: 4.3486 | Train Acc: 74.08% | Val Acc: 67.95%
Epoch 3/50 | Train Loss: 2.7151 | Train Acc: 82.97% | Val Acc: 74.83%
Epoch 4/50 | Train Loss: 1.7075 | Train Acc: 88.86% | Val Acc: 77.60%
Epoch 5/50 | Train Loss: 1.1438 | Train Acc: 92.86% | Val Acc: 80.32%
Epoch 6/50 | Train Loss: 0.8663 | Train Acc: 94.71% | Val Acc: 80.73%
Epoch 7/50 | Train Loss: 0.7182 | Train Acc: 95.67% | Val Acc: 81.27%
Epoch 8/50 | Train Loss: 0.6282 | Train Acc: 96.04% | Val Acc: 81.15%
Epoch 9/50 | Train Loss: 0.5653 | Train Acc: 96.42% | Val Acc: 81.58%
Epoch 10/50 | Train Loss: 0.5261 | Train Acc: 96.51% | Val Acc: 81.73%
Epoch 11/50 | Train Loss: 0.4904 | Train Acc: 96.73% | Val Acc: 81.63%
Epoch 12/50 | Train Loss: 0.4693 | Train Acc: 96.81% | Val Acc: 81.57%
Epoch 13/50 | Train Loss: 0.4545 | Train Acc: 96.77% | Val Acc: 81.88%
Epoch 14/50 | Train Loss: 0.4359 | Train Acc: 96.93% | Val Acc: 82.24%
Epoch 15/50 | T

In [24]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 80.89%
Student Model Test Accuracy Before Pruning: 81.12%


## 95% Sparsity

In [28]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [29]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9548)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [30]:
# Calculate sparsity
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Sparsity: 95.00%


In [31]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 10.6850 | Train Acc: 58.70% | Val Loss: 1.4316 | Val Acc: 65.03% | Sparsity: 95.00%
Epoch 2/50 | Train Loss: 6.3954 | Train Acc: 72.23% | Val Loss: 1.3559 | Val Acc: 66.67% | Sparsity: 95.00%
Epoch 3/50 | Train Loss: 4.9305 | Train Acc: 77.77% | Val Loss: 1.3522 | Val Acc: 69.74% | Sparsity: 95.00%
Epoch 4/50 | Train Loss: 4.0091 | Train Acc: 81.79% | Val Loss: 1.0482 | Val Acc: 73.48% | Sparsity: 95.00%
Epoch 5/50 | Train Loss: 3.3857 | Train Acc: 84.29% | Val Loss: 0.9331 | Val Acc: 75.49% | Sparsity: 95.00%
Epoch 6/50 | Train Loss: 2.8571 | Train Acc: 86.69% | Val Loss: 1.0688 | Val Acc: 73.91% | Sparsity: 95.00%
Epoch 7/50 | Train Loss: 2.5355 | Train Acc: 88.22% | Val Loss: 0.9483 | Val Acc: 75.83% | Sparsity: 95.00%
Epoch 8/50 | Train Loss: 2.2160 | Train Acc: 89.69% | Val Loss: 0.9426 | Val Acc: 76.11% | Sparsity: 95.00%
Epoch 9/50 | Train Loss: 1.9381 | Train Acc: 91.02% | Val Loss: 0.8907 | Val Acc: 76.92% | Sparsity: 95.00%
Epoch 10/50 | Train Loss: 1

In [32]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 74.51%


In [33]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [34]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9548)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [35]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 3.7030 | Train Acc: 63.30% | Val Loss: 1.0019 | Val Acc: 71.73% | Sparsity: 95.00%
Epoch 2/50 | Train Loss: 2.0013 | Train Acc: 77.30% | Val Loss: 0.8983 | Val Acc: 74.82% | Sparsity: 95.00%
Epoch 3/50 | Train Loss: 1.5589 | Train Acc: 81.88% | Val Loss: 0.8323 | Val Acc: 76.34% | Sparsity: 95.00%
Epoch 4/50 | Train Loss: 1.2744 | Train Acc: 85.11% | Val Loss: 0.7953 | Val Acc: 77.41% | Sparsity: 95.00%
Epoch 5/50 | Train Loss: 1.0817 | Train Acc: 87.69% | Val Loss: 0.7935 | Val Acc: 77.64% | Sparsity: 95.00%
Epoch 6/50 | Train Loss: 0.9280 | Train Acc: 89.70% | Val Loss: 0.7879 | Val Acc: 77.64% | Sparsity: 95.00%
Epoch 7/50 | Train Loss: 0.8239 | Train Acc: 91.05% | Val Loss: 0.7739 | Val Acc: 78.31% | Sparsity: 95.00%
Epoch 8/50 | Train Loss: 0.7425 | Train Acc: 92.15% | Val Loss: 0.7726 | Val Acc: 78.63% | Sparsity: 95.00%
Epoch 9/50 | Train Loss: 0.6870 | Train Acc: 92.98% | Val Loss: 0.7781 | Val Acc: 78.10% | Sparsity: 95.00%
Epoch 10/50 | Train Loss: 0.

In [36]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 74.68%


# 90% of Sparsity

In [40]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [41]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9045)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [42]:
# Calculate sparsity
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Sparsity: 90.00%


In [43]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 7.1786 | Train Acc: 70.32% | Val Loss: 1.0627 | Val Acc: 73.05% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 4.0027 | Train Acc: 81.20% | Val Loss: 0.9014 | Val Acc: 76.13% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 2.8617 | Train Acc: 86.29% | Val Loss: 0.7774 | Val Acc: 79.15% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 2.1505 | Train Acc: 89.70% | Val Loss: 0.7498 | Val Acc: 80.77% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 1.7008 | Train Acc: 91.73% | Val Loss: 0.6604 | Val Acc: 82.53% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 1.4243 | Train Acc: 93.20% | Val Loss: 0.6397 | Val Acc: 83.03% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 1.2408 | Train Acc: 93.67% | Val Loss: 0.5997 | Val Acc: 83.67% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 1.0960 | Train Acc: 94.33% | Val Loss: 0.5972 | Val Acc: 84.04% | Sparsity: 90.00%
Epoch 9/50 | Train Loss: 1.0120 | Train Acc: 94.83% | Val Loss: 0.5971 | Val Acc: 83.17% | Sparsity: 90.00%
Epoch 10/50 | Train Loss: 0.

In [44]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 77.19%


In [45]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [46]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9045)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [47]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 2.0202 | Train Acc: 78.16% | Val Loss: 0.6264 | Val Acc: 82.18% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 1.0700 | Train Acc: 87.28% | Val Loss: 0.5675 | Val Acc: 83.89% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 0.7777 | Train Acc: 91.07% | Val Loss: 0.5155 | Val Acc: 85.51% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 0.6310 | Train Acc: 93.00% | Val Loss: 0.4943 | Val Acc: 85.98% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 0.5329 | Train Acc: 94.16% | Val Loss: 0.4814 | Val Acc: 86.17% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 0.4772 | Train Acc: 94.94% | Val Loss: 0.4748 | Val Acc: 86.44% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 0.4376 | Train Acc: 95.41% | Val Loss: 0.4793 | Val Acc: 86.29% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 0.4050 | Train Acc: 95.73% | Val Loss: 0.4648 | Val Acc: 86.92% | Sparsity: 90.00%
Epoch 9/50 | Train Loss: 0.3782 | Train Acc: 95.96% | Val Loss: 0.4678 | Val Acc: 86.64% | Sparsity: 90.00%
Epoch 10/50 | Train Loss: 0.

In [48]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 78.49%


## 79% Sparsity

In [52]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [53]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.7940)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [54]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 79.00%


In [56]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5677 | Train Acc: 93.30% | Val Loss: 0.3121 | Val Acc: 90.96% | Sparsity: 79.00%
Epoch 2/50 | Train Loss: 0.4287 | Train Acc: 94.98% | Val Loss: 0.2946 | Val Acc: 91.59% | Sparsity: 79.00%
Epoch 3/50 | Train Loss: 0.3578 | Train Acc: 95.80% | Val Loss: 0.2879 | Val Acc: 91.86% | Sparsity: 79.00%
Epoch 4/50 | Train Loss: 0.3189 | Train Acc: 96.19% | Val Loss: 0.2836 | Val Acc: 91.93% | Sparsity: 79.00%
Epoch 5/50 | Train Loss: 0.2939 | Train Acc: 96.58% | Val Loss: 0.2834 | Val Acc: 91.79% | Sparsity: 79.00%
Epoch 6/50 | Train Loss: 0.2766 | Train Acc: 96.69% | Val Loss: 0.2820 | Val Acc: 91.76% | Sparsity: 79.00%
Epoch 7/50 | Train Loss: 0.2633 | Train Acc: 96.86% | Val Loss: 0.2767 | Val Acc: 92.17% | Sparsity: 79.00%
Epoch 8/50 | Train Loss: 0.2531 | Train Acc: 96.91% | Val Loss: 0.2771 | Val Acc: 92.04% | Sparsity: 79.00%
Epoch 9/50 | Train Loss: 0.2441 | Train Acc: 96.98% | Val Loss: 0.2785 | Val Acc: 91.89% | Sparsity: 79.00%
Epoch 10/50 | Train Loss: 0.

In [57]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 79.83%


In [58]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [59]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.7940)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [61]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 3.8782 | Train Acc: 82.80% | Val Loss: 0.7457 | Val Acc: 80.33% | Sparsity: 79.00%
Epoch 2/50 | Train Loss: 2.3137 | Train Acc: 88.77% | Val Loss: 0.5443 | Val Acc: 84.38% | Sparsity: 79.00%
Epoch 3/50 | Train Loss: 1.4973 | Train Acc: 92.50% | Val Loss: 0.4454 | Val Acc: 87.43% | Sparsity: 79.00%
Epoch 4/50 | Train Loss: 1.1119 | Train Acc: 94.21% | Val Loss: 0.3978 | Val Acc: 88.54% | Sparsity: 79.00%
Epoch 5/50 | Train Loss: 0.9282 | Train Acc: 94.94% | Val Loss: 0.3716 | Val Acc: 89.32% | Sparsity: 79.00%
Epoch 6/50 | Train Loss: 0.8212 | Train Acc: 95.21% | Val Loss: 0.3758 | Val Acc: 89.67% | Sparsity: 79.00%
Epoch 7/50 | Train Loss: 0.7597 | Train Acc: 95.44% | Val Loss: 0.3552 | Val Acc: 89.77% | Sparsity: 79.00%
Epoch 8/50 | Train Loss: 0.6975 | Train Acc: 95.67% | Val Loss: 0.3539 | Val Acc: 89.57% | Sparsity: 79.00%
Epoch 9/50 | Train Loss: 0.6544 | Train Acc: 95.75% | Val Loss: 0.3422 | Val Acc: 90.38% | Sparsity: 79.00%
Epoch 10/50 | Train Loss: 0.

In [62]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 79.90%


## 59% Sparsity

In [91]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [92]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.5945)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [93]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 59.15%


In [94]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 2.0488 | Train Acc: 90.42% | Val Loss: 0.4354 | Val Acc: 87.52% | Sparsity: 59.15%
Epoch 2/50 | Train Loss: 1.3177 | Train Acc: 93.22% | Val Loss: 0.2955 | Val Acc: 91.79% | Sparsity: 59.15%
Epoch 3/50 | Train Loss: 0.9176 | Train Acc: 94.91% | Val Loss: 0.2731 | Val Acc: 92.04% | Sparsity: 59.15%
Epoch 4/50 | Train Loss: 0.7416 | Train Acc: 95.43% | Val Loss: 0.2603 | Val Acc: 92.54% | Sparsity: 59.15%
Epoch 5/50 | Train Loss: 0.6517 | Train Acc: 95.69% | Val Loss: 0.2538 | Val Acc: 92.93% | Sparsity: 59.15%
Epoch 6/50 | Train Loss: 0.5966 | Train Acc: 95.82% | Val Loss: 0.2468 | Val Acc: 92.98% | Sparsity: 59.15%
Epoch 7/50 | Train Loss: 0.5503 | Train Acc: 95.94% | Val Loss: 0.2444 | Val Acc: 92.88% | Sparsity: 59.15%
Epoch 8/50 | Train Loss: 0.5261 | Train Acc: 96.07% | Val Loss: 0.2418 | Val Acc: 92.96% | Sparsity: 59.15%
Epoch 9/50 | Train Loss: 0.5052 | Train Acc: 96.15% | Val Loss: 0.2437 | Val Acc: 92.98% | Sparsity: 59.15%
Epoch 10/50 | Train Loss: 0.

In [95]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 81.02%


In [96]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [97]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.5945)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [98]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 59.15%


In [99]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5553 | Train Acc: 93.11% | Val Loss: 0.2402 | Val Acc: 93.26% | Sparsity: 59.15%
Epoch 2/50 | Train Loss: 0.3494 | Train Acc: 95.50% | Val Loss: 0.2184 | Val Acc: 93.77% | Sparsity: 59.15%
Epoch 3/50 | Train Loss: 0.2761 | Train Acc: 96.33% | Val Loss: 0.2134 | Val Acc: 93.84% | Sparsity: 59.15%
Epoch 4/50 | Train Loss: 0.2436 | Train Acc: 96.92% | Val Loss: 0.2173 | Val Acc: 93.86% | Sparsity: 59.15%
Epoch 5/50 | Train Loss: 0.2251 | Train Acc: 96.96% | Val Loss: 0.2136 | Val Acc: 93.93% | Sparsity: 59.15%
Epoch 6/50 | Train Loss: 0.2118 | Train Acc: 97.22% | Val Loss: 0.2154 | Val Acc: 93.81% | Sparsity: 59.15%
Epoch 7/50 | Train Loss: 0.2037 | Train Acc: 97.15% | Val Loss: 0.2112 | Val Acc: 93.98% | Sparsity: 59.15%
Epoch 8/50 | Train Loss: 0.1969 | Train Acc: 97.31% | Val Loss: 0.2149 | Val Acc: 93.77% | Sparsity: 59.15%
Epoch 9/50 | Train Loss: 0.1895 | Train Acc: 97.22% | Val Loss: 0.2111 | Val Acc: 93.90% | Sparsity: 59.15%
Epoch 10/50 | Train Loss: 0.

In [100]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 81.32%


## 36% Sparsity

In [70]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [71]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.3618)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [72]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 36.00%


In [73]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.7828 | Train Acc: 91.42% | Val Loss: 0.3494 | Val Acc: 90.29% | Sparsity: 36.00%
Epoch 2/50 | Train Loss: 1.1187 | Train Acc: 94.15% | Val Loss: 0.2728 | Val Acc: 92.15% | Sparsity: 36.00%
Epoch 3/50 | Train Loss: 0.8003 | Train Acc: 95.21% | Val Loss: 0.2440 | Val Acc: 92.93% | Sparsity: 36.00%
Epoch 4/50 | Train Loss: 0.6586 | Train Acc: 95.76% | Val Loss: 0.2380 | Val Acc: 93.02% | Sparsity: 36.00%
Epoch 5/50 | Train Loss: 0.5751 | Train Acc: 95.91% | Val Loss: 0.2373 | Val Acc: 93.14% | Sparsity: 36.00%
Epoch 6/50 | Train Loss: 0.5354 | Train Acc: 96.06% | Val Loss: 0.2275 | Val Acc: 93.33% | Sparsity: 36.00%
Epoch 7/50 | Train Loss: 0.5065 | Train Acc: 96.18% | Val Loss: 0.2262 | Val Acc: 93.49% | Sparsity: 36.00%
Epoch 8/50 | Train Loss: 0.4806 | Train Acc: 96.08% | Val Loss: 0.2197 | Val Acc: 93.59% | Sparsity: 36.00%
Epoch 9/50 | Train Loss: 0.4584 | Train Acc: 96.15% | Val Loss: 0.2270 | Val Acc: 93.41% | Sparsity: 36.00%
Epoch 10/50 | Train Loss: 0.

In [74]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 80.60%


In [75]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [76]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.3618)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [77]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.4725 | Train Acc: 93.86% | Val Loss: 0.2186 | Val Acc: 93.69% | Sparsity: 36.00%
Epoch 2/50 | Train Loss: 0.2966 | Train Acc: 96.01% | Val Loss: 0.2027 | Val Acc: 94.14% | Sparsity: 36.00%
Epoch 3/50 | Train Loss: 0.2346 | Train Acc: 96.89% | Val Loss: 0.1977 | Val Acc: 94.34% | Sparsity: 36.00%
Epoch 4/50 | Train Loss: 0.2076 | Train Acc: 97.14% | Val Loss: 0.1966 | Val Acc: 94.35% | Sparsity: 36.00%
Epoch 5/50 | Train Loss: 0.1925 | Train Acc: 97.36% | Val Loss: 0.1995 | Val Acc: 94.33% | Sparsity: 36.00%
Epoch 6/50 | Train Loss: 0.1835 | Train Acc: 97.47% | Val Loss: 0.1995 | Val Acc: 94.25% | Sparsity: 36.00%
Epoch 7/50 | Train Loss: 0.1772 | Train Acc: 97.51% | Val Loss: 0.1977 | Val Acc: 94.32% | Sparsity: 36.00%
Epoch 8/50 | Train Loss: 0.1724 | Train Acc: 97.52% | Val Loss: 0.1979 | Val Acc: 94.23% | Sparsity: 36.00%
Epoch 9/50 | Train Loss: 0.1697 | Train Acc: 97.55% | Val Loss: 0.1976 | Val Acc: 94.35% | Sparsity: 36.00%
Early stopping triggered at 

In [78]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 81.74%
